### _Section 14.0:_ Load packages

If you haven't installed `gensim` yet, use:
```
conda install gensim
```
- Alternatively, you can use `pip`
- This may require admin privileges

In [1]:
from __future__ import unicode_literals # unicode handling
import codecs
import string
import spacy # for pre-processing and traditional NLP
import numpy as np
import gensim
from gensim import corpora
from gensim.models.word2vec import Word2Vec
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

## _Section 14.1_ - Demo: LDA with `gensim`
### Requires `nltk` be installed!
```
conda install nltk
python -m nltk.downloader all
```
#### Prepare Documents

In [2]:
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]

Cleaning and Preprocessing

In [3]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 

Prepare Document-Term matrix

In [4]:
# Creating the term dictionary of our corpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Run LDA Model

In [5]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Training LDA model on the document term matrix
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

ldamodel.show_topics(num_topics=3, num_words=3)

[(0, '0.064*"driving" + 0.064*"sister" + 0.064*"father"'),
 (1, '0.057*"pressure" + 0.056*"father" + 0.056*"sister"'),
 (2, '0.076*"sugar" + 0.075*"lifestyle" + 0.075*"health"')]

- Each line is a topic with individual topic terms and weights
- In this case, one topic can be termed as 'Bad Health', whereas another can be termed as 'Family'

## _Section 14.2_ - Parsing tweets with spacy
### Write a function that can take a take a sentence parsed by `spacy` and identify if it mentions a company named 'Google' 
- Remember, `spacy` can find entities and codes them as `ORG` if they are a company
- Look at the [slides for class 13](https://github.com/ga-students/DS-SF-44/blob/master/lessons/lesson-13/13-natural-language-processing-and-text-classification.pdf) if you need a hint

#### Bonus (1b)

Parameterize the company name so that the function works for _any company_

In [6]:
# Loading the tweet data
filename = './datasets/captured-tweets.txt'
tweets = []
for tweet in codecs.open(filename, 'r', encoding="utf-8"):
    tweets.append(tweet)
    
sub_tweets = tweets[:2000]

## Load spacy
nlp_toolkit = spacy.load('en')

In [7]:
def mentions_company(parsed):
    for entity in parsed.ents:
        if entity.text == "Google" and entity.label_ == 'ORG':
            return True
    return False

# 1b
def mentions_company(parsed, company='Google'):
    for entity in parsed.ents:
        if entity.text == company and entity.label_ == 'ORG':
            return True
    return False

### Exercise 1c:

Write a function that can take a sentence parsed by `spacy` 
and return the verbs of the sentence (preferably lemmatized)

In [8]:
def get_actions(parsed):
    actions = []
    for el in parsed:
        if el.pos == spacy.parts_of_speech.VERB:
            actions.append(el.text)
    return actions

In [9]:
parsed = nlp_toolkit(tweets[0])
print(tweets[0])
print(parsed[0])
print(parsed[1])
print(parsed)

I made a(n) Small Tourmaline in Paradise Island! https://t.co/cAoW1b6DRc #Gameinsight #Androidgames #Android

I
made
I made a(n) Small Tourmaline in Paradise Island! https://t.co/cAoW1b6DRc #Gameinsight #Androidgames #Android



### Exercise 1d:
For each tweet, parse it using spacy and print it out if the tweet has 'release' __*or*__ 'announce' as a verb
- You'll need to use your `mentions_company` and `get_actions` functions

In [10]:
for i, tweet in enumerate(sub_tweets):
    parsed = nlp_toolkit(tweet)
    if mentions_company(parsed, 'Google'):
        actions = get_actions(parsed)
        if 'release' in actions or 'announce' in actions:
            print("tweet #%d: %s" % (i, tweet))

tweet #1573: Google and Ford to announce partnership on self-driving cars at CES - Fudzilla (blog) https://t.co/6woe56G22Q



### Exercise 1e:
Write a function that identifies countries 
- **Hint**: the entity label for countries is 'GPE' (or _GeoPolitical Entity_)

In [11]:
def mentions_country(parsed, country):
    for entity in parsed.ents:
        if entity.text == country and entity.label_ == 'GPE':
            return True
    return False

### Exercise 1f:

Modify the code from **1d** to now find country tweets that discuss 'Iran' and announcing or releasing

In [12]:
for tweet in tweets:
    parsed = nlp_toolkit(tweet)

    if mentions_country(parsed, 'Iran'):
        actions = get_actions(parsed)
        if 'release' in actions or 'announce' in actions:
            print(tweet)

RT @cerenomri: "Literally every US ally in Mideast is on brink of hot war w/ Iran, so we're going to release $100 billion to Iran this mont…

GOBE! Iran warns Nigeria to release Shiite leader El-Zakzaky - SEE https://t.co/TRshnC6sVU

GOBE! Iran warns Nigeria to release Shiite leader El-Zakzaky - SEE https://t.co/SlvcQtk3vE

RT @cerenomri: "Literally every US ally in Mideast is on brink of hot war w/ Iran, so we're going to release $100 billion to Iran this mont…

Hhmmm. Iran claiming to have 'warned Nigeria' to release detained Shiite leader.... @afalli

RT @cerenomri: "Literally every US ally in Mideast is on brink of hot war w/ Iran, so we're going to release $100 billion to Iran this mont…



## _Section 14.3_ - Build a word2vec model of tweets with gensim
- First take the collection of tweets and tokenize them using spacy

### Exercise 2a:
* Think about how this should be done
* Should you only use upper-case or lower-case? 
* Should you remove punctuations or symbols? 
* Explore the example below

In [13]:
t = tweets[0]

text_split_ex = []
for x in nlp_toolkit(t):
        if x.pos != spacy.parts_of_speech.VERB:
            text_split_ex.append(x.text)
        else:
            text_split_ex.append(x.lemma_)

print(t)
print(text_split_ex)

I made a(n) Small Tourmaline in Paradise Island! https://t.co/cAoW1b6DRc #Gameinsight #Androidgames #Android

['I', 'make', 'a(n', ')', 'Small', 'Tourmaline', 'in', 'Paradise', 'Island', '!', 'https://t.co/cAoW1b6DRc', '#', 'Gameinsight', '#', 'Androidgames', '#', 'Android', '\n']


Run again, including all of the data (*slow*)

In [14]:
text_split = [[x.text if x.pos != spacy.parts_of_speech.VERB else x.lemma_ 
                for x in nlp_toolkit(t)] for t in tweets]

### Exercise 2b:
- Build a `word2vec` model
- Test the window size as well
    - this is how many surrounding words need to be used to model a word   
- What do you think is appropriate for Twitter? 

In [15]:
model = Word2Vec(text_split, size=100, window=32, min_count=5, workers=4)

### Exercise 2c:
Test your word2vec model with a few similarity functions 
* Find words similar to 'Syria'
* Find words similar to 'war'
* Find words similar to 'Iran'
* Find words similar to 'Verizon'

In [16]:
model.most_similar(positive=['Syria'])

/Users/dat-44pt/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Ramazan', 0.9992814660072327),
 ('Ahmad', 0.9991410970687866),
 ('@4FreedominIran', 0.9987801313400269),
 ('@iran_policy', 0.9979690313339233),
 ('Iran', 0.9977648854255676),
 ('opposition', 0.9976183772087097),
 ('regime', 0.9974225759506226),
 ('Hamid', 0.9966427683830261),
 ('democratic', 0.9965959191322327),
 ('Saleh', 0.9964749813079834)]

### Exercise 2d:

Adjust the choices / parameters in (b) and (c) as necessary

## _Section 14.4_ - Tweet filtering exercises

Filter tweets to those that mention 'Iran' or similar entities and 'war' or similar entities
* Do this using just spacy
* Do this using word2vec similarity scores

In [17]:
# Using spacy
for tweet in tweets:
    parsed = nlp_toolkit(tweet)
    if mentions_country(parsed, 'Iran') or mentions_country(parsed, 'Iraq'): # ... you could add more
        if 'attack' in get_actions(parsed):
            print(tweet)

In [18]:
# Using word2vec similarity scores
for tweet in tweets:
    parsed = nlp_toolkit(tweet)

    similarity_to_iran = max([0]+[model.wv.similarity('Iran', tok.text) for tok in parsed if tok.text in model.wv.vocab])
    similarity_to_war = max([0]+[model.wv.similarity('war', tok.text) for tok in parsed if tok.text in model.wv.vocab])
    
    if similarity_to_iran > 0.997 and similarity_to_war > 0.9:
        print(similarity_to_iran, similarity_to_war, tweet)
#        print(tweet)

0.9974129547259368 0.9875673570310093 I made a(n) Small Tourmaline in Paradise Island! https://t.co/cAoW1b6DRc #Gameinsight #Androidgames #Android

0.9974129547259368 0.9934673748659176 Top Android Apps (without all the games) revealed in hidden Google Play Store link -… https://t.co/7ZSs9MkcLm #Android #India

0.9974129547259368 0.9967071947591322 @ImbruedJoint Hector had softly tilted his head towards Stuart in a confused sort of way. "What's Google?" He couldn't possibly know what --

0.9974129547259368 0.9967503232139372 RT @FortuneMagazine: Here’s how Google is taking on Uber in 2016 https://t.co/X5STl6P5Xh

0.9974129547259368 0.9938019894354826 @luckyberry96 Go to this website. https://t.co/4O42egV2MV copy and paste it in twitter. Tweet more for #TeamSeventeen

0.9974129547259368 0.9967503232139372 RT @KunalChoudhary_: Pathankot attack: Youth Congress members burn Sharifs effigy in Bhopal https://t.co/n3uftphdsx (@IndiaToday)

0.9974129547259368 0.9905581658823075 Best colour for

0.9974129547259368 0.9945900199420327 Google News UK: Gun battles rage near Indian consulate in Afghan city - https://t.co/n1zsrjoU3w: Hindu... https://t.co/UyP3yemW9Q #News #UK

0.9974129547259368 0.9897946974492364 Gunbattle Underway Near Indian Consulate in Afghanistan - New York Times

0.9974129547259368 0.9967503232139372 RT @personasama: Was trying to find reference for Saitama's apartment in OPM and I found the actual building: https://t.co/zeyGb4TWEf https…

0.9974129547259368 0.9933656029003854 I'm now an E-List celebrity in Kim Kardashian: Hollywood. You can be famous too by playing on Android! https://t.co/mzXiwNaa45

0.9974129547259368 0.9918507460168422 Reload all tabs in Google Chrome with "Reload All Tabs" extension https://t.co/wOJkkixiyg

0.9974129547259368 0.9880261715319949 I will get your book or ebook on google in 24 hours https://t.co/RMp6r3m9K5 https://t.co/ewudcmyFHd

0.9974129547259368 0.9934673748659176 Top Android Apps (without all the games) revealed in hidd

0.9974129547259368 0.9875673570310093 #Is this a very SERIOUS misunderstanding of Key in Java Google App Engine Datastore?

0.9974129547259368 0.9967503232139372 RT @Iowa4Bernie: Do you need a ride to the caucuses in Iowa? Can you make the time to come here and help? This is the next step!

0.9974129547259368 0.9945679335455999 "''Use Internet Explorer''- Frank and Rai whispered in my left ear, ''Use Google Chrome'' - Baam whispered in my right ear"

0.9974129547259368 0.9875673570310093 Privacy-as-a-Service Scatters Data in Disappearing Clouds - https://t.co/sm4Kr2O60z

0.9974129547259368 0.9897946974492364 Gunbattle Underway Near Indian Consulate in Afghanistan - New York Times.. https://t.co/ztO7ey2rv6 #news

0.9974129547259368 0.9875673570310093 Aussie dies in snowmobile crash #au https://t.co/2vspVkJ4HY https://t.co/rAIZ6CC1lq

0.9974129547259368 0.9967071947591322 That is not her in the pic. That's Coco from Coco &amp; Ice T! Put that pic in Google and see what name pops up. SMH 

0.9974129547259368 0.9875673570310093 #Is this a very SERIOUS misunderstanding of Key in Java Google App Engine Datastore?

0.9974129547259368 0.9967503232139372 RT @VeryOldPics: The First Google Team in 1999. https://t.co/MXUHzjtgsI

0.9974129547259368 0.9967503232139372 RT @GuptaKushal0588: @UpGrad_Edu Google's headquarters back in 1998. What a journey!!! https://t.co/dON9dhkTIB

0.9974129547259368 0.9967503232139372 RT @KimHanBin_INTL: [!!!] Don't forget to VOTE again today for iKON's DUMB&amp;DUMBER in MCountdown using your Google accounts! https://t.co/Co…

0.9974129547259368 0.9917306999749711 #OnThisDay in #history a white slaver just sold a slave. https://t.co/tcVngc2xgQ

0.9974129547259368 0.9875673570310093 google search: cusswords in kids movies

0.9974129547259368 0.9967503232139372 RT @zirigozacom: Brands That Won (and Lost) #Google in 2015 - https://t.co/P22LkAjYar by https://t.co/P09iWmCbo2 #PymesUnidas https://t.co/…

0.9974129547259368 0.9967503232139372 RT @corysnearo

0.9974129547259368 0.9886086987675294 Leaking gas well in Porter Ranch lacked a working safety valve - Los Angeles Times  https://t.co/zDIyRDeSXT

0.9999999999999999 0.9943839811564072 #出会い #無料 #セフレ #メル友 Saudi Arabia cuts ties with Iran - Mail &amp; Guardian Online  https://t.co/9s0dtpAJnl

0.9974129547259368 0.9967071947591322 Cops: Man fires shot in parking garage dispute on New Year's - San Francisco Chronicle  https://t.co/xK3CZjLTXp

0.9974129547259368 0.9967503232139372 RT @zirigozacom: Brands That Won (and Lost) #Google in 2015 - https://t.co/P22LkAjYar by https://t.co/P09iWmCbo2 #PymesUnidas https://t.co/…

0.9974129547259368 0.9962163625609814 Google Glass 2.0 comes in a folding version - wwntradio https://t.co/6cUhoKpxTb https://t.co/AuxmZr3bY5

0.9974129547259368 0.9903052650916683 Why does every Google image search I do have porn in it? I mean totally innocent searches and *boom* porn. Glad I'm in to that kind of stuff

0.9974129547259368 0.9967503232139372 RT @stephen_mcd:

...

0.997764820294419 0.9967503232139372 RT @4FreedominIran: #Syria: Over 55,000 people including 2,500 children killed in 2015 https://t.co/7CBc2ZtuEp https://t.co/Ad6D8ckgp3 #Ira…

0.9999999999999999 0.9967503232139372 RT @iran_policy: Ahmad Ramazan: #Iran regime's Al-Alam satellite TV broadcasts live from Damascus &amp; has direct ties to Assad's office. #Syr…

0.9999999999999999 0.9989075547608625 RT @iran_policy: Saleh Hamid: #Iran regime's meddling in #Syria increases its internal bleeding. Rouhani failed to deliver on any of his pr…

0.9999999999999999 0.9967503232139372 RT @4FreedominIran: .@sanabarghzahedi: State news agency IRNA admits that 400 of #Iran regime's forces have been killed in #Syria https://t…

0.9999999999999999 0.9967503232139372 RT @iran_policy: Ahmad Ramazan: #Iran regime's Al-Alam satellite TV broadcasts live from Damascus &amp; has direct ties to Assad's office. #Syr…

0.9999999999999999 0.9967503232139372 RT @boghche: List of women executed under Rouhani regim

0.9999999999999999 0.9967503232139372 RT @4FreedominIran: Saleh Hamid: Severing of Arab states' ties with #Iran regime benefits Iranian &amp; #Syria opposition movements. #SaudiArab…

0.9999999999999999 0.994548095734564 Tehran must "act like a normal country" before Saudi restores diplomatic relations, says Saudi FM https://t.co/vHZTTIDMSR #SaudiArabia #Iran

0.9999999999999999 0.9967503232139372 RT @NIACouncil: "#SaudiArabia’s destabilizing activities are vindication of nuclear deal it struck w/ #Iran in 2015." #IranDeal https://t.c…

0.9974129547259368 0.9916761352096828 In case you're wondering where we're staying in Israel. https://t.co/9APmnrZn1X

0.9999999999999999 0.9956949335482228 @saudgo @Reuters  yes! the saudis (backed by Israel) just want to show their "power" after the Iran nuclear deal.

0.9974129547259368 0.9967503232139372 RT @realDonaldTrump: Hillary Clinton said that it is O.K. to ban Muslims from Israel by building a WALL, but not O.K. to do so in the U.S. …

0.997

0.9999999999999999 0.9967503232139372 RT @aliamir1342: Rouhani is absolutely responsible for crime against humanity in #CampLiberty #Iran #HumanRights #Europe #US #UK https://t.…

0.9999999999999999 0.9999999999999998 RT @iran_policy: .@sanabarghzahedi: ... so #Iran IRGC Quds Force commander Qassem Soleimani went to Moscow to seek help in #Syria war https…

0.9999999999999999 0.9967503232139372 RT @Maryam_Rajavi: Killing under torture, death penalties &amp; harassing #Sunni prisoners in #Iran ‘s prisons 4 protesting against the regime …

0.9999999999999999 0.995231668727408 Top #USA House Democrat ‘disappointed’ at #Iran sanctions delay. https://t.co/T58mhyf54L #News #MTP #FNS #Obama

0.9999999999999999 0.9999999999999998 RT @iran_policy: Ahmad Ramazan: #Iran regime no longer calls the shots in #Syria war. It has been superceded by Russia. #No2Rouhani #StopEx…

0.9999999999999999 0.99352539777142 UN chief calls Saudi break in ties with Iran "deeply worrying" https://t.co/CJTPAd5QFf  ht

0.9999999999999999 0.9967503232139372 RT @4FreedominIran: Ahmad Ramazan: #Iran regime recently saw it was losing grip of #Syria. With Russia's entry, regime opened 10 fronts in …

0.9999999999999999 0.9970412607833222 RT @iran_policy: .@sanabarghzahedi: Our info says Putin-Khamenei agreed in meeting that #Iran IRGC sends in ground forces full force &amp; Russ…

0.9999999999999999 0.9945659382272876 List of women executed under Rouhani regime! https://t.co/5qUfsU05jD #humanRights #No2Rouhani #Iran #IranTalks https://t.co/fZrK2tiAYg

0.9999999999999999 0.9967503232139372 RT @RossMorley: What a great #Greenwald piece - enjoy! https://t.co/YuCIe47kyN #Privacy #espionage #NSA #Congress #Israel #Iran #Obama #Hoe…

0.9999999999999999 0.9989075547608625 RT @MicahZenko: Iran can retaliate by closing its airspace to Emirates and Etihad...logistics nightmare. https://t.co/nhJorXnXPZ https://t.…

0.9999999999999999 0.9967503232139372 RT @farmandeMehrdad: #Bahrain slams hypocritical comments by #Ir

0.9974129547259368 0.9967503232139372 RT @produk718: Produk has a show on 01/06/2016 at 01:00 PM @ Microsoft Theater in Los Angeles, CA https://t.co/pWsCIWQtGo #concert

0.9974129547259368 0.9875673570310093 https://t.co/XcEsvTQH14 RALLY Gameplay, Sign in the channel and let that favorite like to support https://t.co/JuYMLx6jjB

0.9974129547259368 0.9967503232139372 RT @PCMag: Tesla Model S catches fire at supercharger station in Norway: https://t.co/5vUldlxmHB #tesla #cars https://t.co/Vgj4XCt3cK

0.9974129547259368 0.9967071947591322 @indexmodel thought part of the reason for today's sell off might be due to the tesla fire in norway https://t.co/caC7itSk3T

0.9974129547259368 0.9967503232139372 RT @latimes: Tesla delivers more cars than ever before: 17,400 in the fourth quarter https://t.co/MuJOIzdGYq https://t.co/IyftqchAuS

